# 0. 초기 환경 셋팅

In [1]:
import time
import math
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import pandas as pd

from PIL import Image
from numpy import asarray

from pathlib import Path

from torchvision import models

# from tqdm import tqdm
from functools import lru_cache


print(torch.__version__)
print(pd.__version__)

1.6.0+cu101
1.1.2


In [2]:
# set device
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device: {}".format(device))

device: cuda


In [3]:
# for reproducibility
seed = 0

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

# 1. DATA Check

In [4]:
data_train = pd.read_csv("./train.tsv", sep='\t', header=None)
print(data_train)

data_test = pd.read_csv("./test.tsv", sep='\t', header=None)
print(data_test)

                    0   1   2
0        3_5_1123.jpg   3   5
1       3_20_1048.jpg   3  20
2         4_2_401.jpg   4   2
3         4_7_740.jpg   4   7
4         4_11_93.jpg   4  11
...               ...  ..  ..
15995  13_15_1600.jpg  13  15
15996  13_16_1570.jpg  13  16
15997   13_17_986.jpg  13  17
15998  13_18_4980.jpg  13  18
15999   13_20_282.jpg  13  20

[16000 rows x 3 columns]
             0
0        0.jpg
1        1.jpg
2        2.jpg
3        3.jpg
4        4.jpg
...        ...
3992  3992.jpg
3993  3993.jpg
3994  3994.jpg
3995  3995.jpg
3996  3996.jpg

[3997 rows x 1 columns]


# 2. Dataset & DataLoader

In [5]:
combined_label = {
    (3, 5): 0,
    (3, 20): 1,
    (4, 2): 2,
    (4, 7): 3,
    (4, 11): 4,
    (5, 8): 5,
    (7, 1): 6,
    (7, 20): 7,
    (8, 6): 8,
    (8, 9): 9,
    (10, 20): 10,
    (11, 14): 11,
    (13, 1): 12,
    (13, 6): 13,
    (13, 9): 14,
    (13, 15): 15,
    (13, 16): 16,
    (13, 17): 17,
    (13, 18): 18,
    (13, 20): 19
}

index2label = {v: k for k, v in combined_label.items()}

In [6]:
class PlantImageDataset(Dataset):
    
    def __init__(self, tsv_path, data_path):
        self.data_path = Path(data_path)
        self.data = self._preprocess(tsv_path)
            
    def _preprocess(self, tsv_path):
        data = pd.read_csv(tsv_path, sep='\t', header=None)
        return data
    
    def __len__(self):
        return len(self.data)
    
    # Override 필수 함수
    @lru_cache(maxsize=100000)
    def __getitem__(self, index):
        file_name = Path(self.data.iloc[index][0])
        
        if str(self.data_path) == 'train':
            plant_label = self.data.iloc[index][1]
            disease_label = self.data.iloc[index][2]
        else:
            plant_label = -1
            disease_label = -1
        
        file_path = self.data_path / file_name
        
        data = Image.open(file_path)
        data = asarray(data) # (H, W, C) 
        data = torch.FloatTensor(data)
        data = data.permute(2, 0, 1).contiguous()  # (H, W, C) --> (C, H, W)
        
        # print(data, plant_label, disease_label)
        
        label = combined_label.get((plant_label, disease_label), -1)
        
        if str(self.data_path) == 'train' and label == -1:
            raise Exception('Label Error!!')
        
        return data, torch.tensor(label)

In [7]:
train_dataset = PlantImageDataset(tsv_path="./train.tsv", data_path="train")


train_size = int(len(train_dataset) * 0.8)
valid_size = int(len(train_dataset) * 0.2)
assert len(train_dataset) == (train_size + valid_size)

train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])
print(len(train_dataset), len(valid_dataset))

test_dataset = PlantImageDataset(tsv_path="./test.tsv", data_path="test")

12800 3200


In [8]:
batch_size = 16
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=False)

valid_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          drop_last=False)

test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          drop_last=False)

In [9]:
# for x, y in test_loader:
#     print(x, y)
#     break

# 3. Model

In [10]:
class PlantModel(nn.Module):
    def __init__(self, label_size=20):
        super(PlantModel, self).__init__()

        self.pre_model = models.resnext101_32x8d()
        self.pre_model.fc = nn.Linear(2048, label_size)

    def forward(self, x):
        out = self.pre_model(x)
        
        return out

model = PlantModel()
model = model.to(device)
print(model)

PlantModel(
  (pre_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): 

In [11]:
# model = models.wide_resnet50_2()
# model.fc = nn.Linear(2048, 14)

# model = model.to(device)

# print(model)

In [12]:
# model = models.vgg16()
# model.classifier._modules['6'] = nn.Linear(4096, 14)
# # model.fc = nn.Linear(1000, 14)

# model = model.to(device)

# print(model)

# 4. Train

In [13]:
# optimizer
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

criterion = nn.CrossEntropyLoss(reduction='sum').to(device)

In [14]:
def train_one_epoch(model, train_loader, criterion, optimizer=None, device=None, epoch=None):
    model.train() # 학습 모드
    
    total_loss = 0.
    total_num = 0
    
    start_time = time.time()
   
    for batch, (data, label) in enumerate(train_loader):
        # data
        data = data.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        hyp = model(data)
        
        loss = criterion(hyp, label)
        
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()

        total_loss += loss.detach().item()
        total_num += 1
        
        log_interval = 100
        if batch % log_interval == 0 and batch >= 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | grad_norm {:5.2f} | loss {:5.2f} | time {:.4f}'.format(
                    epoch+1, batch+1, len(train_loader), grad_norm, loss.detach().item(), elapsed))
            start_time = time.time()
    
    train_loss = total_loss / total_num
    return train_loss
        
        
def validate_one_epoch(model, valid_loader, criterion, optimizer=None, device=None, epoch=None, print_output=False):
    model.eval() # 평가 모드
    
    total_loss = 0.
    total_num = 0
    
    correct= 0
    total_size = 0
   
    with torch.no_grad():
        for batch, (data, label) in enumerate(valid_loader):
            # data
            data = data.to(device)
            label = label.to(device)
            
            hyp = model(data)
        
            loss = criterion(hyp, label)
            
            if print_output:
                hyp = torch.argmax(hyp, dim=-1)
                correct += (hyp == label).sum().item()

            total_loss += loss.detach().item()
            total_num += 1
            total_size += data.size(0)
    
    valid_loss = total_loss / total_num
    accuracy = correct / total_size
    
    return valid_loss, accuracy

def save_model():
    pass

In [ ]:
training_epochs = 50 # 100

for epoch in range(training_epochs):
    
    # train_one_epoch
    train_loss = train_one_epoch(model, train_loader, criterion,  optimizer=optimizer, device=device, epoch=epoch)
    print("| epoch {:3d} | train_loss : {:7.4f}".format(epoch+1, train_loss))
    
    # validate_one_epoch
    valid_loss, accuracy = validate_one_epoch(model, valid_loader, criterion, optimizer=None, device=device, epoch=epoch, print_output=True)
    print("| epoch {:3d} | valid_loss : {:7.4f} accuracy : {:.4f}".format(epoch+1, valid_loss, accuracy))
    
    # save_model
    # save_model()
    
    lr_scheduler.step(valid_loss)

/home/ubuntu/anaconda3/envs/TF2_37/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)


| epoch   1 |     1/  800 batches | grad_norm 11285.97 | loss 50.25 | time 0.5461
| epoch   1 |   101/  800 batches | grad_norm 16.48 | loss 36.90 | time 47.8221
| epoch   1 |   201/  800 batches | grad_norm 27.52 | loss 31.85 | time 48.5100
| epoch   1 |   301/  800 batches | grad_norm 15.38 | loss 28.99 | time 46.8379
| epoch   1 |   401/  800 batches | grad_norm 21.30 | loss 40.28 | time 46.5221
| epoch   1 |   501/  800 batches | grad_norm 16.35 | loss 26.43 | time 46.7530
| epoch   1 |   601/  800 batches | grad_norm 20.25 | loss 42.77 | time 46.4373
| epoch   1 |   701/  800 batches | grad_norm 19.02 | loss 27.01 | time 46.8288
| epoch   1 | train_loss : 38.3598
| epoch   1 | valid_loss : 42.7795 accuracy : 0.2264
| epoch   2 |     1/  800 batches | grad_norm 25.06 | loss 33.62 | time 0.4271
| epoch   2 |   101/  800 batches | grad_norm 23.05 | loss 30.28 | time 41.8917
| epoch   2 |   201/  800 batches | grad_norm 21.45 | loss 23.02 | time 41.2943
| epoch   2 |   301/  800 batch

# 5. Inference

In [ ]:
result_p = []
result_d = []

for batch, (data, label) in enumerate(test_loader):
    # data
    data = data.to(device)
    label = label.to(device)
    
    hyp = model(data)
    
    pred = torch.argmax(hyp, dim=-1)
    
    sample_num = pred.size(0)
    for i in range(sample_num):
        pred_idx = pred[i].detach().item()
        
        pred_plant, pred_disease = index2label[pred_idx]
        result_p.append(pred_plant)
        result_d.append(pred_disease)
        
# print(result)

In [ ]:
data_test = pd.read_csv("./test.tsv", sep='\t', header=None)
# print(data_test)

data_test[1] = result_p
data_test[2] = result_d
print(data_test)

data_test.to_csv("test_result.tsv", index=False, header=None, sep="\t")